<a href="https://colab.research.google.com/github/anmolpandeybtech/Skin_Disease_Hackathon/blob/main/Skin_Disease_resnet152V2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from IPython.display import Javascript

def disable_colab_idle_timeout():
    js_code = '''
    function ClickConnect(){
        console.log("Working");
        document.querySelector("colab-toolbar-button#connect").click()
    }
    setInterval(ClickConnect,60000)
    '''
    display(Javascript(js_code))

disable_colab_idle_timeout()

<IPython.core.display.Javascript object>

In [2]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [3]:
!kaggle datasets download -d ascanipek/skin-diseases

skin-diseases.zip: Skipping, found more recently modified local copy (use --force to force download)


In [5]:
import zipfile
zip_ref = zipfile.ZipFile('/content/skin-diseases.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [6]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten
from tensorflow.keras.models import Model
from tensorflow.keras.applications.inception_v3 import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.preprocessing.image import ImageDataGenerator,load_img
from tensorflow.keras.models import Sequential
import numpy as np
from glob import glob
from tensorflow.keras.callbacks import ModelCheckpoint


In [7]:
IMAGE_SIZE = [224, 224]

train_path = '/content/kaggle/train'
valid_path = '/content/kaggle/val'

In [8]:
import tensorflow
resnet152V2 =tensorflow.keras.applications.ResNet152V2(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

In [9]:
for layer in resnet152V2.layers:
    layer.trainable = False

In [10]:
folders = glob('/content/kaggle/train/*')

In [11]:
x = Flatten()(resnet152V2.output)

In [12]:
prediction = Dense(len(folders), activation='softmax')(x)

model = Model(inputs=resnet152V2.input, outputs=prediction)

In [13]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 224, 224, 3)]        0         []                            
                                                                                                  
 conv1_pad (ZeroPadding2D)   (None, 230, 230, 3)          0         ['input_1[0][0]']             
                                                                                                  
 conv1_conv (Conv2D)         (None, 112, 112, 64)         9472      ['conv1_pad[0][0]']           
                                                                                                  
 pool1_pad (ZeroPadding2D)   (None, 114, 114, 64)         0         ['conv1_conv[0][0]']          
                                                                                              

In [20]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [21]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

In [22]:
training_set = train_datagen.flow_from_directory('/content/kaggle/train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 30909 images belonging to 6 classes.


In [23]:
validation_set = train_datagen.flow_from_directory('/content/kaggle/val',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

Found 3923 images belonging to 6 classes.


In [24]:
test_set = test_datagen.flow_from_directory('/content/kaggle/test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 3928 images belonging to 6 classes.


In [25]:
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard

# Define your callbacks
checkpoint_callback = ModelCheckpoint(filepath='model_checkpoint.h5', save_best_only=True)
early_stopping_callback = EarlyStopping(patience=5, restore_best_weights=True)
tensorboard_callback = TensorBoard(log_dir='./logs')

In [26]:
history = model.fit_generator(
    training_set,
    validation_data=validation_set,
    epochs=20,
    steps_per_epoch=len(training_set),
    validation_steps=len(validation_set),
    callbacks=[checkpoint_callback, early_stopping_callback, tensorboard_callback]
)

<ipython-input-26-0d0bbca2bb51>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  history = model.fit_generator(


Epoch 1/20
966/966 [==============================] - ETA: 0s - loss: 7.4436 - accuracy: 0.5913

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


966/966 [==============================] - 717s 722ms/step - loss: 7.4436 - accuracy: 0.5913 - val_loss: 7.9426 - val_accuracy: 0.6238
Epoch 2/20
966/966 [==============================] - 672s 696ms/step - loss: 6.7469 - accuracy: 0.6771 - val_loss: 9.0693 - val_accuracy: 0.6378
Epoch 3/20
966/966 [==============================] - 665s 688ms/step - loss: 6.3299 - accuracy: 0.7097 - val_loss: 8.7955 - val_accuracy: 0.6628
Epoch 4/20
966/966 [==============================] - 664s 687ms/step - loss: 5.9567 - accuracy: 0.7336 - val_loss: 9.1788 - val_accuracy: 0.6592
Epoch 5/20
966/966 [==============================] - 667s 690ms/step - loss: 5.7038 - accuracy: 0.7495 - val_loss: 8.5884 - val_accuracy: 0.6839
Epoch 6/20
966/966 [==============================] - 691s 715ms/step - loss: 5.4550 - accuracy: 0.7634 - val_loss: 9.3451 - val_accuracy: 0.7058
